Just wanting to see what would happen if we use fine-tuned bert itself to find out the topic rather then we defined anchor text.

In [3]:
import geopandas as gpd
import numpy as np
import pandas as pd
import ast
from shapely.geometry import Polygon
from shapely.geometry import Point

In [4]:
# merge the gentri label to the vectorization text
# use 'polygon' or 'wgs84_polygon', depend on what coordinate the label is
# load gentri label and description
df_london_all3 = pd.read_csv("data/cleaned_projects_raw.csv")
lsoa_label = gpd.read_file("data/gentri_data/london_gentri_labeled_25.shp")
lsoa_label.crs

<Projected CRS: EPSG:27700>
Name: OSGB36 / British National Grid
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: United Kingdom (UK) - offshore to boundary of UKCS within 49°45'N to 61°N and 9°W to 2°E; onshore Great Britain (England, Wales and Scotland). Isle of Man onshore.
- bounds: (-9.01, 49.75, 2.01, 61.01)
Coordinate Operation:
- name: British National Grid
- method: Transverse Mercator
Datum: Ordnance Survey of Great Britain 1936
- Ellipsoid: Airy 1830
- Prime Meridian: Greenwich

In [5]:
# turn polygon coordinate into shapely 
# make sure to drop the null geodata
text_valid = df_london_all3[
    df_london_all3['wgs84_polygon.coordinates'].notnull() & 
    (df_london_all3['wgs84_polygon.type'] == 'Polygon')
].copy()
text_valid["geometry"] = text_valid["wgs84_polygon.coordinates"].apply(
    lambda coords: Polygon(ast.literal_eval(coords)[0])
)
# creat GeoDataFrame， set WGS84 coordinate
gdf_text = gpd.GeoDataFrame(text_valid, geometry="geometry", crs="EPSG:4326")

In [6]:
gdf_27700 = gdf_text.to_crs("EPSG:27700")
# get the centroid of each polygon
gdf_27700["centroid"] = gdf_27700.geometry.centroid
joined_centroid = gpd.sjoin(gdf_27700.set_geometry("centroid"), lsoa_label[["geometry", "gentrified"]],how="left",predicate="within")
joined_cleaned = joined_centroid[joined_centroid["gentrified"].notna()].copy()

In [7]:
joined_cleaned.head(5)

,site_name,decision_date,valid_date,polygon.geometries,polygon.type,wgs84_polygon.coordinates,wgs84_polygon.type,description,borough,street_name,total_no_proposed_residential_units,total_no_existing_residential_units,polygon,wgs84_polygon,polygon.coordinates,year,geometry,centroid,index_right,gentrified
0,NaN,2015-04-17,2015-02-20,[],geometrycollection,"[[[-0.330153, 51.480892], [-0.330154, 51.48086...",Polygon,Erection of side and rear extension to existin...,Hounslow,Roxborough Avenue,NaN,NaN,NaN,NaN,NaN,2015,"POLYGON ((516053.478 177092.461, 516053.477 17...",POINT (516051.984 177090.981),2620.0,False
1,NaN,2015-03-12,2015-01-19,"[{'coordinates': [[[532400.5928219, 185252.991...",GeometryCollection,"[[[-0.0917876, 51.5506417], [-0.0918015, 51.55...",Polygon,"Side extension at basement, rebuild steps, new...",Islington,Highbury New Park,NaN,NaN,NaN,NaN,NaN,2015,"POLYGON ((532402.417 185253.256, 532401.522 18...",POINT (532395.532 185252.542),2716.0,False
2,NaN,2015-07-27,2015-06-01,[],geometrycollection,"[[[-0.414458, 51.439358], [-0.414459, 51.43933...",Polygon,Certificate of Lawfulness for the erection of ...,Hounslow,Parkfield Road,NaN,NaN,NaN,NaN,NaN,2015,"POLYGON ((510299.53 172343.489, 510299.525 172...",POINT (510298.015 172342.015),2556.0,False
3,NaN,2015-11-30,2015-10-16,"[{'coordinates': [[[530957.946236, 184135.3910...",GeometryCollection,"[[[-0.1129983, 51.5409351], [-0.1130422, 51.54...",Polygon,Erection of single storey timber clad garden r...,Islington,Hemingford Road,NaN,NaN,NaN,NaN,NaN,2015,"POLYGON ((530959.741 184135.694, 530956.693 18...",POINT (530946.671 184138.641),2639.0,False
4,20,2015-07-28,2015-06-17,"[{'coordinates': [[[521146.4472015, 180763.396...",GeometryCollection,"[[[-0.2555649, 51.5128182], [-0.2556564, 51.51...",Polygon,Single storey (max 6m deep and max 3.875m high...,Ealing,Bowes Road,NaN,NaN,NaN,NaN,NaN,2015,"POLYGON ((521148.041 180763.566, 521141.701 18...",POINT (521140.311 180783.762),1222.0,False


In [25]:
df = joined_cleaned[['description', 'gentrified']].dropna()
df = df.rename(columns={'description': 'text', 'gentrified': 'label'})
df['label'] = df['label'].astype(int)

In [9]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)

In [10]:
dataset = dataset.train_test_split(test_size=0.2, seed=42)

In [11]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/123645 [00:00<?, ? examples/s]

Map:   0%|          | 0/30912 [00:00<?, ? examples/s]

In [18]:
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)

import evaluate

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels),
        "f1": f1.compute(predictions=predictions, references=labels, average="binary")
    }

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

e:\dissertation\project\housing_feature_gentrification\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling 

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bert_gentri_output",
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    max_steps=500,
)

In [20]:
from transformers import Trainer

small_train = tokenized_dataset["train"].shuffle(seed=42).select(range(1000))
small_eval = tokenized_dataset["test"].shuffle(seed=42).select(range(200))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_eval,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

C:\Users\hp\AppData\Local\Temp\ipykernel_19828\211151094.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,0.275800
100,0.194400
150,0.219000
200,0.198600
250,0.194600
300,0.227300
350,0.156400
400,0.144500
450,0.221800
500,0.138600


TrainOutput(global_step=500, training_loss=0.19711716365814208, metrics={'train_runtime': 5056.6879, 'train_samples_per_second': 0.791, 'train_steps_per_second': 0.099, 'total_flos': 529869594624000.0, 'train_loss': 0.19711716365814208, 'epoch': 4.0})

In [21]:
trainer.evaluate()

e:\dissertation\project\housing_feature_gentrification\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.2532597780227661,
 'eval_accuracy': {'accuracy': 0.945},
 'eval_f1': {'f1': 0.0},
 'eval_runtime': 26.1812,
 'eval_samples_per_second': 7.639,
 'eval_steps_per_second': 0.955,
 'epoch': 4.0}

In [26]:
import torch
from sklearn.utils.class_weight import compute_class_weight
labels = df['label']
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float)

In [27]:
import torch.nn as nn
class WeightedDistilBert(DistilBertForSequenceClassification):
    def __init__(self, config, class_weights):
        super().__init__(config)
        self.class_weights = class_weights

    def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
        outputs = super().forward(input_ids=input_ids, attention_mask=attention_mask, labels=None, **kwargs)
        logits = outputs.logits

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(weight=self.class_weights.to(logits.device))
            loss = loss_fct(logits, labels)
            return {"loss": loss, "logits": logits}
        else:
            return {"logits": logits}

In [34]:
from transformers import DistilBertConfig

config = DistilBertConfig.from_pretrained("distilbert-base-uncased", num_labels=2)
model = WeightedDistilBert.from_pretrained(
    "distilbert-base-uncased", 
    num_labels=2,
    class_weights=class_weights
)

Some weights of WeightedDistilBert were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
training_args = TrainingArguments(
    output_dir="./bert_gentri_output",
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    max_steps=500,
)

In [35]:
small_train = tokenized_dataset["train"].shuffle(seed=42).select(range(1000))
small_eval = tokenized_dataset["test"].shuffle(seed=42).select(range(200))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_eval,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

C:\Users\hp\AppData\Local\Temp\ipykernel_19828\2724630097.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
e:\dissertation\project\housing_feature_gentrification\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TypeError: DistilBertForSequenceClassification.forward() got an unexpected keyword argument 'num_items_in_batch'